In [1]:
import sympy
from sympy.plotting import plot
import sympy as sp
import penbegone as pbg
from penbegone import common as bgcom
from penbegone import plotting as bgplot
from penbegone.common import printeq

%matplotlib inline
%load_ext blackcellmagic
%load_ext autoreload
%autoreload 2

# ArduPilot TECS mathematical model

This is just a simplification.

Nomenclature:
- $h$: Height, $m$
- $\dot{h}$: Climb rate, $m/s$
- $V$: Airspeed, $m/s$
- $V_d$: Demanded airspeed, $m/s$
- $\dot{V}$: Airspeed derivative, $m/s^2$

Parameters:
- $V_{max}$: Maximum airspeed, $m/s$
- $V_{min}$: Minimum airspeed, $m/s$
- $\dot{h}_{c,max}$: Maximum climb rate, $m/s$
- $\dot{h}_{s,max}$: Maximum sink rate, $m/s$
- $\dot{h}_{s,min}$: Minimum sink rate, $m/s$

Constants:
- $g$: Acceleration of gravity, $m/s^2$

In [2]:
t = sp.symbols("t", real=True)
h, h_dem, v, v_dem = bgcom.functions("h h_d v v_dem", t)
h_dot, v_c, v_dot = sp.symbols("hdot v_cruise vdot")
v_max, v_min, climb_max, sink_max, sink_min = sp.symbols("V_max V_min hdot_cmax hdot_smax hdot_smin")
thr_max, thr_min, thr_trim = sp.symbols("t_{max} t_{min} t_{trim}")
w, k_tc, k_pd, k_i, k_td = sp.symbols("w k_tc k_pd k_i k_td", real=True)
g = sp.symbols("g", real=True)

h_dot_ = sp.diff(h, t) 
v_dot_ = sp.diff(v, t)

In [3]:
STEdot_max, STEdot_min, STEdot_neg_max = sp.symbols("Edot^*_max Edot^*_min Edot^*_nmax")

STEdot_max_ = g*climb_max
STEdot_min_ = -g*sink_min
STEdot_neg_max = -g*sink_max

In [4]:
## _update_speed_demand

# Perhaps the increase of _TAS_dem when at max sink plays a role?

vel_rate_max_ = 0.5*STEdot_max_/v
vel_rate_neg_cruise = 0.9*STEdot_min_/v_c
vel_rate_neg_max = 0.9*STEdot_min_/v_max

# I can add the LPF effect of TIME_CONST here, instead of pure differentiation.
vdot_dem_ = sp.diff(v_dem, t)

In [5]:
## _update_height_demand

# Can add the LPF effect of HDEM_TCONST here, over h_dem.
h_dot_dem_ = sp.diff(h_dem)


In [6]:
## _update_energies

SPE_dem_ = h_dem*g
SKE_dem_ = 0.5*v_dem*v_dem
SKE_est_ = 0.5*v*v
SKE_dot_dem_= v*vdot_dem_ # In the source vdot_dem_ is high-passed. How does this translate to the actual system?
SKE_dot_ = v*v_dot # Same comment as above.
SPE_est_ = h*g
SPE_dot_ = h_dot*g

In [7]:
## _update_pitch

w_k_ = w
w_p_ = 2-w

SEB_dem_ = SPE_dem_*w_p_ - SKE_dem_*w_k_
SEB_est_ = SPE_est_*w_p_ - SKE_est_*w_k_
SEB_err_ = SEB_dem_ - SEB_est_
SEB_dot_est_ = SPE_dot_*w_p_ - SKE_dot_*w_k_

SEB_dot_dem_ = SEB_err_/k_tc + h_dot_dem_*g*w_p_
SEB_dot_err_ = SEB_dot_dem_ - SEB_dot_est_
SEB_dot_dem_tot_ = SEB_dot_dem_ + SEB_dot_err_*k_pd

# Adding integrator states.
SEB_dot_int = bgcom.functions("x_{intsebdot}", t)
SEB_dot_int_dot_ = SEB_dot_err_*k_i
KE_int = bgcom.functions("x_{intKE}", t)
KE_int_dot_ = (SKE_est_ - SKE_dem_)*w_k_/k_tc

pitch_dem_ = (SEB_dot_dem_tot_ + SEB_dot_int + KE_int) / (v * g)

In [8]:
## _update_throttle_with_airspeed

STE_error_ = SKE_dem_ - SKE_est_ + SPE_dem_ - SPE_est_
SPE_dot_dem_ = (SPE_dem_ - SPE_est_)/k_tc
STE_dot_dem_ = SPE_dot_dem_ + SKE_dot_dem_
STE_dot_err_ = STE_dot_dem_ - SKE_dot_ - SPE_dot_

K_thr2STE_ = (STEdot_max_ - STEdot_min_)/(thr_max - thr_min)
K_STE2Thr_ = 1 / (K_thr2STE_ * k_tc)
# Adding integrator state.
integTHR_state = bgcom.functions("x_{intthr}",t)
integTHR_state_dot_ = STE_error_*k_i*K_STE2Thr_
ff_throttle = STE_dot_dem_/K_thr2STE_ + thr_trim

throttle_dem_ = K_STE2Thr_*(STE_error_ + STE_dot_err_*k_td) + ff_throttle + integTHR_state

In [9]:
throttle_dem_.simplify()

(g*k_tc**2*(hdot_cmax + hdot_smin)*(t_{trim} + x_{intthr}(t)) + k_tc*(t_{max} - t_{min})*(-g*h(t) + g*h_d(t) + k_tc*v(t)*Derivative(v_dem(t), t)) - (t_{max} - t_{min})*(k_tc*(g*h(t) - g*h_d(t) + 0.5*v(t)**2 - 0.5*v_dem(t)**2) + k_td*(g*h(t) - g*h_d(t) + k_tc*(g*hdot + vdot*v(t) - v(t)*Derivative(v_dem(t), t)))))/(g*k_tc**2*(hdot_cmax + hdot_smin))

## Kinematic model

In [10]:
pitch, throttle, m = sp.symbols("theta delta_t m")
F_t, c_drag, S, rho = sp.symbols("F_t C_drag S rho")
theta_0 = sp.symbols("theta_0")
drag_ = 0.5*S*rho*v*v*c_drag
f_v_dot_ = 1/m*(sp.sin(pitch)*g + F_t*throttle - drag_)
f_h_dot_ = v*sp.sin(pitch-theta_0)  # Perhaps add effect of increasing lift as airspeed increases?

## System definition

In [11]:
fcl_1 = f_v_dot_.subs({pitch: pitch_dem_, throttle: throttle_dem_})
fcl_2 = f_h_dot_.subs({pitch: pitch_dem_})

In [12]:
# fcl_1.simplify()

In [13]:
# fcl_2.simplify()

### Transformation onto trim

In [14]:
v_star, v_dem_star, h_star = sp.symbols("v^* v^*_dem h^*")
fclt_1 = fcl_1.subs({v: v_star+v_c, v_dem: v_dem_star+v_c})
fclt_2 = fcl_2.subs({v: v_star+v_c, v_dem: v_dem_star+v_c})

In [15]:
# fclt_1.simplify()

In [16]:
# fclt_2.simplify()

### Is origin a solution?

In [17]:
fclt_1_trim = fclt_1.subs({v_star: 0, v_dot: 0, h: 0, h_dot: 0, v_dem_star: 0, h_dem: 0}).simplify().factor()
x_int_pitch = sp.symbols("x_ipitch")
fclt_1_trim = fclt_1_trim.subs({KE_int/(g*v_c)+SEB_dot_int/(g*v_c): x_int_pitch/(g*v_c)})
fclt_1_trim

1.0*(-0.5*C_drag*S*rho*v_cruise**2 + 1.0*F_t*t_{trim} + 1.0*F_t*x_{intthr}(t) + 1.0*g*sin(x_ipitch/(g*v_cruise)))/m

In [18]:
fclt_2_trim = fclt_2.subs({v_star: 0, v_dot: 0, h: 0, v_dem_star: 0, h_dem: 0, h_dot: 0, h_dot_dem_: 0, v_dot_: 0}).simplify()
fclt_2_trim = fclt_2_trim.subs({KE_int+SEB_dot_int: x_int_pitch})
fclt_2_trim

v_cruise*sin((-g*theta_0*v_cruise + x_ipitch)/(g*v_cruise))

Not quite. The integrators state needs to be calculated.

In [19]:
x_int_trims = sp.solve([fclt_1_trim, fclt_2_trim], x_int_pitch, integTHR_state, dict=True)
x_int_pitch_trim = x_int_trims[0][x_int_pitch]
integTHR_state_trim = x_int_trims[0][integTHR_state]
x_int_pitch_trim, integTHR_state_trim


(g*theta_0*v_cruise,
 (0.5*C_drag*S*rho*v_cruise**2 - F_t*t_{trim} - g*sin(theta_0))/F_t)